In [1]:
#imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
#########################
#Preliminary Exploration#
#########################

In [3]:
#prepare paths to import data
msa_csv = "../data/msa_data.csv"
national_csv = "../data/national_data.csv"
skill_csv = "../data/skill_data.csv"
posting_csv = "../data/job_postings.csv"

In [4]:
#read msa_data as df and check
msa_df = pd.read_csv(msa_csv, sep=",")
msa_df.head()

,MSA_ID,Metro Area,OCC_CODE,Job Group,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
0,10180,"Abilene, TX",11-1021,General and Operations Managers,1100.0,16.499,1.01,84000.0,22810.0,50350.0,73850.0,106870.0,154520.0
1,10180,"Abilene, TX",11-2022,Sales Managers,110.0,1.728,0.63,124710.0,47360.0,72140.0,119920.0,155050.0,207570.0
2,10180,"Abilene, TX",11-2030,Public Relations and Fundraising Managers,40.0,0.571,1.06,NaN,NaN,NaN,NaN,NaN,NaN
3,10180,"Abilene, TX",11-3010,Administrative Services and Facilities Managers,150.0,2.197,1.05,84350.0,52360.0,58760.0,71810.0,91380.0,120490.0
4,10180,"Abilene, TX",11-3021,Computer and Information Systems Managers,50.0,0.819,0.28,116390.0,56260.0,74910.0,104350.0,144370.0,194760.0


In [5]:
#Break off separate 
msa_Only_DF = msa_df[['MSA_ID', 'Metro Area']]
msa_Only_DF.head()

,MSA_ID,Metro Area
0,10180,"Abilene, TX"
1,10180,"Abilene, TX"
2,10180,"Abilene, TX"
3,10180,"Abilene, TX"
4,10180,"Abilene, TX"


In [6]:
# #Make a separate DF to preserve  Metro Area Text
msa_Only_DF = msa_Only_DF.drop_duplicates(ignore_index = True, subset='MSA_ID')
msa_Only_DF

,MSA_ID,Metro Area
0,10180,"Abilene, TX"
1,10380,"Aguadilla-Isabela, PR"
2,10420,"Akron, OH"
3,10500,"Albany, GA"
4,10540,"Albany, OR"
...,...,...
391,76900,"Portsmouth, NH-ME"
392,77200,"Providence-Warwick, RI-MA"
393,78100,"Springfield, MA-CT"
394,78700,"Waterbury, CT"


In [7]:
msa_Only_DF = msa_Only_DF.reset_index(drop = True)
msa_Only_DF

,MSA_ID,Metro Area
0,10180,"Abilene, TX"
1,10380,"Aguadilla-Isabela, PR"
2,10420,"Akron, OH"
3,10500,"Albany, GA"
4,10540,"Albany, OR"
...,...,...
391,76900,"Portsmouth, NH-ME"
392,77200,"Providence-Warwick, RI-MA"
393,78100,"Springfield, MA-CT"
394,78700,"Waterbury, CT"


In [8]:
#check msa_df datatypes
msa_df.dtypes

MSA_ID                    int64
Metro Area               object
OCC_CODE                 object
Job Group                object
Total Employment        float64
Jobs per 1000           float64
Location Quotient       float64
Mean Annual Wage        float64
Annual Wage 10th Pct    float64
Annual Wage 25th Pct    float64
Median Annual Wage      float64
Annual Wage 75th Pct    float64
Annual Wage 90th Pct    float64
dtype: object

In [9]:
#check NaNs
msa_df.isnull().sum().sort_values(ascending = False)

Location Quotient       11651
Jobs per 1000           11651
Total Employment        11651
Annual Wage 90th Pct     6458
Annual Wage 75th Pct     4361
Median Annual Wage       3508
Annual Wage 25th Pct     2918
Annual Wage 10th Pct     2692
Mean Annual Wage         2688
Job Group                   0
OCC_CODE                    0
Metro Area                  0
MSA_ID                      0
dtype: int64

In [10]:
#calculate percentage of NaN
msa_length = msa_df["OCC_CODE"].count()
msa_NaN = msa_df["Total Employment"].isnull().sum()
msa_Nan_Form = msa_NaN/msa_length*100
print(f"NaN values are {msa_Nan_Form}")

NaN values are 8.735913143234185


In [11]:
#read national_data as df and check
national_df = pd.read_csv(national_csv, sep=",")
national_df.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Total Employment (National),Mean Annual Wage (National),Annual Wage 10th Pct (National),Annual Wage 25th Pct (National),Median Annual Wage (National),Annual Wage 75th Pct (National),Annual Wage 90th Pct (National),Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage
0,Computer Operator,43-9011.00,64.9,35.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High school diploma or equivalent,None,False,False,True,False
1,Business Intelligence Analyst,15-1199.93,43.8,56.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bachelor's degree,None,False,True,False,False
2,IT Project Manager,15-1199.95,43.8,56.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bachelor's degree,None,False,True,False,False
3,Network Engineer / Architect,15-1199.02,43.8,56.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bachelor's degree,None,False,True,False,False
4,Network Engineer / Architect,15-1133.00,14.7,85.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bachelor's degree,None,False,True,False,False


In [12]:
#check national_df datatypes
national_df.dtypes

BLS Job Title                               object
LONG_OCC_CODE                               object
Percentage with AA Degree or Less          float64
Percentage with BA Degree or More          float64
Total Employment (National)                float64
Mean Annual Wage (National)                float64
Annual Wage 10th Pct (National)            float64
Annual Wage 25th Pct (National)            float64
Median Annual Wage (National)              float64
Annual Wage 75th Pct (National)            float64
Annual Wage 90th Pct (National)            float64
Typical education needed for entry          object
Work experience in a related occupation     object
Non-Entry Level                               bool
BA Entry Level                                bool
Middle Skilled                                bool
Low Wage                                      bool
dtype: object

In [13]:
#check NaNs
national_df.isnull().sum().sort_values(ascending = False)

Annual Wage 90th Pct (National)            232
Annual Wage 75th Pct (National)            166
Median Annual Wage (National)              165
Annual Wage 25th Pct (National)            165
Annual Wage 10th Pct (National)            165
Mean Annual Wage (National)                165
Total Employment (National)                164
Percentage with AA Degree or Less            6
Work experience in a related occupation      6
Typical education needed for entry           6
Percentage with BA Degree or More            6
LONG_OCC_CODE                                0
Low Wage                                     0
Middle Skilled                               0
Non-Entry Level                              0
BA Entry Level                               0
BLS Job Title                                0
dtype: int64

In [14]:
#calculate percentage of NaN
national_length = national_df["LONG_OCC_CODE"].count()
national_NaN = national_df["Annual Wage 90th Pct (National)"].isnull().sum()
print(f"NaN values are {national_NaN/national_length*100}")

NaN values are 40.98939929328622


In [15]:
#read skill_data as df and check
skill_df = pd.read_csv(skill_csv, sep=",")
skill_df.head(20)

,LONG_OCC_CODE,BLS Job Title,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,...,Persistence_im,Self Control_im,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex
0,11-1011.00,Chief Executives,2.67,5.33,7.00,2.00,1.33,3.67,1.00,2.12,...,4.43,4.52,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33
1,11-1011.03,Chief Sustainability Officers,2.67,4.33,7.00,4.33,1.00,2.33,1.00,1.88,...,4.31,4.00,3.35,4.08,6.67,6.67,6.00,5.00,3.33,6.33
2,11-1021.00,General and Operations Managers,1.00,3.67,7.00,1.33,1.33,3.33,2.00,2.12,...,4.04,4.31,3.57,4.33,5.33,6.00,5.67,6.33,4.67,6.00
3,11-1031.00,Legislators,3.67,3.00,7.00,3.67,1.00,4.67,NaN,NaN,...,NaN,NaN,NaN,NaN,5.33,5.00,5.00,5.67,4.00,4.33
4,11-2011.00,Advertising and Promotions Managers,5.33,4.67,7.00,2.00,1.67,2.33,1.38,1.75,...,4.32,4.18,3.70,4.35,5.33,5.33,5.33,5.00,4.00,5.33
5,11-2011.01,Green Marketers,4.33,3.00,5.33,5.33,1.00,2.33,NaN,NaN,...,NaN,NaN,NaN,NaN,5.33,5.33,5.00,5.00,4.00,4.83
6,11-2021.00,Marketing Managers,3.67,5.33,7.00,2.33,1.00,2.67,1.12,1.88,...,4.23,3.87,3.80,4.01,6.00,5.67,5.33,5.67,5.00,6.17
7,11-2022.00,Sales Managers,2.00,4.67,7.00,2.00,3.00,3.67,1.38,2.00,...,4.48,4.30,3.83,4.52,5.33,5.67,4.67,4.00,5.33,5.50
8,11-2031.00,Public Relations and Fundraising Managers,5.00,3.67,7.00,1.33,1.33,3.67,1.00,1.75,...,4.53,3.94,4.15,4.17,5.33,6.00,5.00,6.00,4.67,6.17
9,11-3011.00,Administrative Services Managers,1.00,5.33,7.00,2.33,2.00,2.67,2.12,2.25,...,3.98,4.53,3.65,4.44,4.67,5.67,4.67,6.33,4.00,4.67


In [16]:
#check skill_df datatypes
skill_df.dtypes

LONG_OCC_CODE             object
BLS Job Title             object
Artistic_oi              float64
Conventional_oi          float64
Enterprising_oi          float64
                          ...   
Independence_ex          float64
Recognition_ex           float64
Relationships_ex         float64
Support_ex               float64
Working Conditions_ex    float64
Length: 352, dtype: object

In [17]:
#check NaNs
skill_df.isnull().sum().sort_values(ascending = False)

Attention to Detail_im         7
Integrity_im                   7
Achievement/Effort_im          7
Adaptability/Flexibility_im    7
Analytical Thinking_im         7
                              ..
Achievement_ex                 0
Independence_ex                0
Recognition_ex                 0
Relationships_ex               0
LONG_OCC_CODE                  0
Length: 352, dtype: int64

In [18]:
#calculate percentage of NaN
skill_length = skill_df["LONG_OCC_CODE"].count()
skill_NaN = skill_df["Concern for Others_im"].isnull().sum()
print(f"NaN values are {skill_NaN/skill_length*100}")

NaN values are 0.7186858316221766


In [19]:
#read job_posting as df and check
posting_df = pd.read_csv(posting_csv, sep=",")
posting_df.head()

,JOB_ID,BLS Job Title,LONG_OCC_CODE,YEAR,STATE,CITY,MSA_ID,CANON_MAXIMUM_DEGREE,CANON_MINIMUM_DEGREE,CANON_OTHER_DEGREES,...,CIP_CODE,STANDARD_MAJOR,MAX_EXPERIENCE,MIN_EXPERIENCE,MAX_ANNUAL_SALARY,MAX_HOURLY_SALARY,MIN_ANNUAL_SALARY,MIN_HOURLY_SALARY,YEARS_OF_EXPERIENCE,CANON_JOB_HOURS
0,38472841335,Recruiter (General),13-1071.91,2019,WA,Seattle,42660.0,Bachelor's,Higher Secondary Certificate,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2+ years|2 years,fulltime
1,38474073426,Estimator,13-1051.00,2019,AZ,Chandler,38060.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38474083740,Auto Body Technician,49-3021.00,2019,AZ,Chandler,38060.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38709992297,General Manager,11-1021.92,2020,FL,Tampa,45300.0,Bachelor's,Higher Secondary Certificate,NaN,...,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,3-5 years,NaN
4,38709918452,Maintenance Helper / Assistant,49-9098.00,2020,TX,San Antonio,41700.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#check posting_df datatypes
posting_df.dtypes

JOB_ID                       int64
BLS Job Title               object
LONG_OCC_CODE               object
YEAR                         int64
STATE                       object
CITY                        object
MSA_ID                     float64
CANON_MAXIMUM_DEGREE        object
CANON_MINIMUM_DEGREE        object
CANON_OTHER_DEGREES        float64
CANON_PREFERRED_DEGREES     object
CANON_REQUIRED_DEGREES      object
CIP_CODE                    object
STANDARD_MAJOR              object
MAX_EXPERIENCE             float64
MIN_EXPERIENCE             float64
MAX_ANNUAL_SALARY          float64
MAX_HOURLY_SALARY          float64
MIN_ANNUAL_SALARY          float64
MIN_HOURLY_SALARY          float64
YEARS_OF_EXPERIENCE         object
CANON_JOB_HOURS             object
dtype: object

In [21]:
#check NaNs
posting_df.isnull().sum().sort_values(ascending = False)

CANON_OTHER_DEGREES        5079
STANDARD_MAJOR             5042
CIP_CODE                   5042
MAX_EXPERIENCE             4926
CANON_PREFERRED_DEGREES    4905
CANON_MAXIMUM_DEGREE       4895
MIN_EXPERIENCE             4662
CANON_REQUIRED_DEGREES     4600
CANON_MINIMUM_DEGREE       4565
YEARS_OF_EXPERIENCE        4146
CANON_JOB_HOURS            4057
MAX_ANNUAL_SALARY          3163
MAX_HOURLY_SALARY          3163
MIN_ANNUAL_SALARY          3163
MIN_HOURLY_SALARY          3163
MSA_ID                       24
CITY                         23
STATE                         0
YEAR                          0
LONG_OCC_CODE                 0
BLS Job Title                 0
JOB_ID                        0
dtype: int64

In [22]:
#calculate percentage of NaN
posting_length = posting_df["LONG_OCC_CODE"].count()
posting_NaN = posting_df["CANON_OTHER_DEGREES"].isnull().sum()
print(f"NaN values are {posting_NaN/posting_length*100}")

NaN values are 100.0


In [23]:
#verify that 5079 is 100% the size of the DF
print(f"Length: {posting_length}. NaN Count: {posting_NaN}.")

Length: 5079. NaN Count: 5079.


In [24]:
print(national_df['Non-Entry Level'].count())
print(skill_df['Artistic_oi'].count())
print(posting_df['YEAR'].count())
print(msa_df['Total Employment'].count())

566
974
5079
121718


In [25]:
######################
#Clean individual DFs#
######################

In [26]:
#Drop NaNs from msa_df, and print difference from old df
msa_df = msa_df.dropna()
print(msa_length-msa_df["OCC_CODE"].count())

17634


In [27]:
#compare greatest NaN row to rows dropped
print(msa_NaN)

11651


In [28]:
#Print new msa_df
msa_df.head()

,MSA_ID,Metro Area,OCC_CODE,Job Group,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
0,10180,"Abilene, TX",11-1021,General and Operations Managers,1100.0,16.499,1.01,84000.0,22810.0,50350.0,73850.0,106870.0,154520.0
1,10180,"Abilene, TX",11-2022,Sales Managers,110.0,1.728,0.63,124710.0,47360.0,72140.0,119920.0,155050.0,207570.0
3,10180,"Abilene, TX",11-3010,Administrative Services and Facilities Managers,150.0,2.197,1.05,84350.0,52360.0,58760.0,71810.0,91380.0,120490.0
4,10180,"Abilene, TX",11-3021,Computer and Information Systems Managers,50.0,0.819,0.28,116390.0,56260.0,74910.0,104350.0,144370.0,194760.0
6,10180,"Abilene, TX",11-3051,Industrial Production Managers,40.0,0.603,0.48,98640.0,57580.0,68030.0,88490.0,111530.0,130800.0


In [29]:
#wage data does not seem to be of importance, and has a high amount of NaN values. Dropped columns
national_df = national_df.drop(columns=['Annual Wage 90th Pct (National)', 'Annual Wage 75th Pct (National)', 'Annual Wage 25th Pct (National)', 'Annual Wage 10th Pct (National)', 'Median Annual Wage (National)', 'Mean Annual Wage (National)', 'Total Employment (National)'])
national_df.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage
0,Computer Operator,43-9011.00,64.9,35.1,High school diploma or equivalent,None,False,False,True,False
1,Business Intelligence Analyst,15-1199.93,43.8,56.4,Bachelor's degree,None,False,True,False,False
2,IT Project Manager,15-1199.95,43.8,56.4,Bachelor's degree,None,False,True,False,False
3,Network Engineer / Architect,15-1199.02,43.8,56.4,Bachelor's degree,None,False,True,False,False
4,Network Engineer / Architect,15-1133.00,14.7,85.2,Bachelor's degree,None,False,True,False,False


In [30]:
#The earlier exploration showed a few columns with about 6 NaN values. Dropped, and checked lengths of old and new dfs
national_df = national_df.dropna()
print(national_df["LONG_OCC_CODE"].count())
print(national_length)

560
566


In [31]:
#skill df had minimum NaNs. Dropped NaNs, and verified minimal data loss
skill_df = skill_df.dropna()
print(skill_df["LONG_OCC_CODE"].count())
print(skill_length)

967
974


In [32]:
#Earlier exploration showed it is best to remove certain rows from posting_df
posting_df = posting_df.filter(['MSA_ID', 'CITY', 'STATE', 'YEAR', 'LONG_OCC_CODE', 'BLS Job Title', 'JOB_ID'])
posting_df.head()

,MSA_ID,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID
0,42660.0,Seattle,WA,2019,13-1071.91,Recruiter (General),38472841335
1,38060.0,Chandler,AZ,2019,13-1051.00,Estimator,38474073426
2,38060.0,Chandler,AZ,2019,49-3021.00,Auto Body Technician,38474083740
3,45300.0,Tampa,FL,2020,11-1021.92,General Manager,38709992297
4,41700.0,San Antonio,TX,2020,49-9098.00,Maintenance Helper / Assistant,38709918452


In [33]:
#######################
#Prepare DFs for Merge#
#######################

In [34]:
#creat OCC_CODE column that drops last 3 chars from LONG_OCC_CODE
national_df['OCC_CODE'] = national_df['LONG_OCC_CODE'].str.slice(stop=7)
national_df.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage,OCC_CODE
0,Computer Operator,43-9011.00,64.9,35.1,High school diploma or equivalent,None,False,False,True,False,43-9011
1,Business Intelligence Analyst,15-1199.93,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
2,IT Project Manager,15-1199.95,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
3,Network Engineer / Architect,15-1199.02,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
4,Network Engineer / Architect,15-1133.00,14.7,85.2,Bachelor's degree,None,False,True,False,False,15-1133


In [35]:
#creat OCC_CODE column that drops last 3 chars from LONG_OCC_CODE
skill_df['OCC_CODE'] = skill_df['LONG_OCC_CODE'].str.slice(stop=7)
skill_df.head()

,LONG_OCC_CODE,BLS Job Title,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,...,Self Control_im,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE
0,11-1011.00,Chief Executives,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,...,4.52,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011
1,11-1011.03,Chief Sustainability Officers,2.67,4.33,7.0,4.33,1.00,2.33,1.00,1.88,...,4.00,3.35,4.08,6.67,6.67,6.00,5.00,3.33,6.33,11-1011
2,11-1021.00,General and Operations Managers,1.00,3.67,7.0,1.33,1.33,3.33,2.00,2.12,...,4.31,3.57,4.33,5.33,6.00,5.67,6.33,4.67,6.00,11-1021
4,11-2011.00,Advertising and Promotions Managers,5.33,4.67,7.0,2.00,1.67,2.33,1.38,1.75,...,4.18,3.70,4.35,5.33,5.33,5.33,5.00,4.00,5.33,11-2011
6,11-2021.00,Marketing Managers,3.67,5.33,7.0,2.33,1.00,2.67,1.12,1.88,...,3.87,3.80,4.01,6.00,5.67,5.33,5.67,5.00,6.17,11-2021


In [36]:
#creat OCC_CODE column that drops last 3 chars from LONG_OCC_CODE
posting_df['OCC_CODE'] = posting_df['LONG_OCC_CODE'].str.slice(stop=7)
posting_df.head()

,MSA_ID,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID,OCC_CODE
0,42660.0,Seattle,WA,2019,13-1071.91,Recruiter (General),38472841335,13-1071
1,38060.0,Chandler,AZ,2019,13-1051.00,Estimator,38474073426,13-1051
2,38060.0,Chandler,AZ,2019,49-3021.00,Auto Body Technician,38474083740,49-3021
3,45300.0,Tampa,FL,2020,11-1021.92,General Manager,38709992297,11-1021
4,41700.0,San Antonio,TX,2020,49-9098.00,Maintenance Helper / Assistant,38709918452,49-9098


In [37]:
#Creat a DF for only Job Titles and LONG_OCC
LONG_OCC_TITLE = posting_df[['LONG_OCC_CODE', 'BLS Job Title']]
LONG_OCC_TITLE.head()

,LONG_OCC_CODE,BLS Job Title
0,13-1071.91,Recruiter (General)
1,13-1051.00,Estimator
2,49-3021.00,Auto Body Technician
3,11-1021.92,General Manager
4,49-9098.00,Maintenance Helper / Assistant


In [38]:
LONG_OCC_TITLE = LONG_OCC_TITLE.drop_duplicates(subset='LONG_OCC_CODE', ignore_index = True)
LONG_OCC_TITLE.head()

,LONG_OCC_CODE,BLS Job Title
0,13-1071.91,Recruiter (General)
1,13-1051.00,Estimator
2,49-3021.00,Auto Body Technician
3,11-1021.92,General Manager
4,49-9098.00,Maintenance Helper / Assistant


In [39]:
#Create a DF for only MSA_ID and Location
msa_CiSt_DF = posting_df[['MSA_ID', 'CITY', 'STATE']]
msa_CiSt_DF.head()

,MSA_ID,CITY,STATE
0,42660.0,Seattle,WA
1,38060.0,Chandler,AZ
2,38060.0,Chandler,AZ
3,45300.0,Tampa,FL
4,41700.0,San Antonio,TX


In [40]:
msa_CiSt_DF = msa_CiSt_DF.drop_duplicates(ignore_index = True)
msa_CiSt_DF.head()

,MSA_ID,CITY,STATE
0,42660.0,Seattle,WA
1,38060.0,Chandler,AZ
2,45300.0,Tampa,FL
3,41700.0,San Antonio,TX
4,41860.0,Dublin,CA


In [41]:
msa_CiSt_DF = msa_CiSt_DF.astype('object')
msa_CiSt_DF.head()

,MSA_ID,CITY,STATE
0,42660,Seattle,WA
1,38060,Chandler,AZ
2,45300,Tampa,FL
3,41700,San Antonio,TX
4,41860,Dublin,CA


In [42]:
#Check remaining datatypes before merge
msa_df.dtypes

MSA_ID                    int64
Metro Area               object
OCC_CODE                 object
Job Group                object
Total Employment        float64
Jobs per 1000           float64
Location Quotient       float64
Mean Annual Wage        float64
Annual Wage 10th Pct    float64
Annual Wage 25th Pct    float64
Median Annual Wage      float64
Annual Wage 75th Pct    float64
Annual Wage 90th Pct    float64
dtype: object

In [43]:
national_df.dtypes

BLS Job Title                               object
LONG_OCC_CODE                               object
Percentage with AA Degree or Less          float64
Percentage with BA Degree or More          float64
Typical education needed for entry          object
Work experience in a related occupation     object
Non-Entry Level                               bool
BA Entry Level                                bool
Middle Skilled                                bool
Low Wage                                      bool
OCC_CODE                                    object
dtype: object

In [44]:
skill_df.dtypes

LONG_OCC_CODE             object
BLS Job Title             object
Artistic_oi              float64
Conventional_oi          float64
Enterprising_oi          float64
                          ...   
Recognition_ex           float64
Relationships_ex         float64
Support_ex               float64
Working Conditions_ex    float64
OCC_CODE                  object
Length: 353, dtype: object

In [45]:
posting_df.dtypes

MSA_ID           float64
CITY              object
STATE             object
YEAR               int64
LONG_OCC_CODE     object
BLS Job Title     object
JOB_ID             int64
OCC_CODE          object
dtype: object

In [46]:
#Perform Groubys on national_df
national_df_OCC = national_df.groupby(['OCC_CODE']).mean()
national_df.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage,OCC_CODE
0,Computer Operator,43-9011.00,64.9,35.1,High school diploma or equivalent,None,False,False,True,False,43-9011
1,Business Intelligence Analyst,15-1199.93,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
2,IT Project Manager,15-1199.95,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
3,Network Engineer / Architect,15-1199.02,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
4,Network Engineer / Architect,15-1133.00,14.7,85.2,Bachelor's degree,None,False,True,False,False,15-1133


In [47]:
national_df_LONG = national_df.groupby(['LONG_OCC_CODE']).mean()
national_df_LONG.head()

,Percentage with AA Degree or Less,Percentage with BA Degree or More,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage
LONG_OCC_CODE,,,,,,
11-1011.00,30.2,69.9,1.0,0.0,0.0,0.0
11-1021.91,54.1,45.9,1.0,0.0,0.0,0.0
11-1021.92,54.1,45.9,1.0,0.0,0.0,0.0
11-2021.91,30.3,69.8,1.0,0.0,0.0,0.0
11-2021.92,30.3,69.8,1.0,0.0,0.0,0.0


In [48]:
#Perform Groubys on skill_df
skill_df_OCC = skill_df.groupby(['OCC_CODE']).mean()
skill_df.head()

,LONG_OCC_CODE,BLS Job Title,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,...,Self Control_im,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE
0,11-1011.00,Chief Executives,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,...,4.52,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011
1,11-1011.03,Chief Sustainability Officers,2.67,4.33,7.0,4.33,1.00,2.33,1.00,1.88,...,4.00,3.35,4.08,6.67,6.67,6.00,5.00,3.33,6.33,11-1011
2,11-1021.00,General and Operations Managers,1.00,3.67,7.0,1.33,1.33,3.33,2.00,2.12,...,4.31,3.57,4.33,5.33,6.00,5.67,6.33,4.67,6.00,11-1021
4,11-2011.00,Advertising and Promotions Managers,5.33,4.67,7.0,2.00,1.67,2.33,1.38,1.75,...,4.18,3.70,4.35,5.33,5.33,5.33,5.00,4.00,5.33,11-2011
6,11-2021.00,Marketing Managers,3.67,5.33,7.0,2.33,1.00,2.67,1.12,1.88,...,3.87,3.80,4.01,6.00,5.67,5.33,5.67,5.00,6.17,11-2021


In [49]:
skill_df_LONG = skill_df.groupby(['LONG_OCC_CODE']).mean()
skill_df.head()

,LONG_OCC_CODE,BLS Job Title,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,...,Self Control_im,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE
0,11-1011.00,Chief Executives,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,...,4.52,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011
1,11-1011.03,Chief Sustainability Officers,2.67,4.33,7.0,4.33,1.00,2.33,1.00,1.88,...,4.00,3.35,4.08,6.67,6.67,6.00,5.00,3.33,6.33,11-1011
2,11-1021.00,General and Operations Managers,1.00,3.67,7.0,1.33,1.33,3.33,2.00,2.12,...,4.31,3.57,4.33,5.33,6.00,5.67,6.33,4.67,6.00,11-1021
4,11-2011.00,Advertising and Promotions Managers,5.33,4.67,7.0,2.00,1.67,2.33,1.38,1.75,...,4.18,3.70,4.35,5.33,5.33,5.33,5.00,4.00,5.33,11-2011
6,11-2021.00,Marketing Managers,3.67,5.33,7.0,2.33,1.00,2.67,1.12,1.88,...,3.87,3.80,4.01,6.00,5.67,5.33,5.67,5.00,6.17,11-2021


In [50]:
#Split dfs by year if I am able to come back to this
posting_2019 = posting_df.loc[posting_df['YEAR'] == 2019]
posting_2019.head(20)

,MSA_ID,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID,OCC_CODE
0,42660.0,Seattle,WA,2019,13-1071.91,Recruiter (General),38472841335,13-1071
1,38060.0,Chandler,AZ,2019,13-1051.00,Estimator,38474073426,13-1051
2,38060.0,Chandler,AZ,2019,49-3021.00,Auto Body Technician,38474083740,49-3021
7,29820.0,Henderson,NV,2019,49-3021.00,Auto Body Technician,38477189379,49-3021
14,19100.0,Richardson,TX,2019,11-2022.00,Sales Manager,38536361414,11-2022
15,19100.0,Plano,TX,2019,51-1011.92,Production Plant Manager,38536316917,51-1011
16,19820.0,Royal Oak,MI,2019,53-7061.00,Detailer,38501127611,53-7061
17,19100.0,Plano,TX,2019,43-6014.00,Administrative Clerk / Coordinator,38538984588,43-6014
26,19100.0,The Colony,TX,2019,13-1032.00,Auto Damage Appraiser,38676824178,13-1032
32,19820.0,Ferndale,MI,2019,49-3021.00,Auto Body Technician,38634526873,49-3021


In [51]:
posting_2020 = posting_df.loc[posting_df['YEAR'] == 2020]
posting_2020.head(20)

,MSA_ID,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID,OCC_CODE
3,45300.0,Tampa,FL,2020,11-1021.92,General Manager,38709992297,11-1021
4,41700.0,San Antonio,TX,2020,49-9098.00,Maintenance Helper / Assistant,38709918452,49-9098
5,41860.0,Dublin,CA,2020,11-1021.92,General Manager,38709946390,11-1021
6,41940.0,Mountain View,CA,2020,49-3021.00,Auto Body Technician,38709947747,49-3021
8,12060.0,Marietta,GA,2020,49-9098.00,Maintenance Helper / Assistant,38714233901,49-9098
9,45300.0,Tampa,FL,2020,49-3021.00,Auto Body Technician,38714234989,49-3021
10,37980.0,Malvern,PA,2020,49-9098.00,Maintenance Helper / Assistant,38713580400,49-9098
11,19100.0,Richardson,TX,2020,49-9098.00,Maintenance Helper / Assistant,38713580614,49-9098
12,19100.0,Arlington,TX,2020,49-3023.00,Automotive Service Technician / Mechanic,38720574509,49-3023
13,19100.0,Irving,TX,2020,49-9098.00,Maintenance Helper / Assistant,38720577681,49-9098


In [52]:
#Perform Groubys on posting_df
posting_df_OCC = posting_df.groupby(['OCC_CODE']).mean()
posting_df_OCC.head()

,MSA_ID,YEAR,JOB_ID
OCC_CODE,,,
11-1011,41860.000000,2020.000000,3.875803e+10
11-1021,28558.316498,2019.597315,3.869620e+10
11-2021,36280.000000,2019.500000,3.862572e+10
11-2022,18628.000000,2019.600000,3.870868e+10
11-3011,19100.000000,2019.500000,3.873672e+10


In [53]:
posting_df_LONG_OCC = posting_df.groupby(['LONG_OCC_CODE']).count()
posting_df_LONG_OCC.head()

,MSA_ID,CITY,STATE,YEAR,BLS Job Title,JOB_ID,OCC_CODE
LONG_OCC_CODE,,,,,,,
11-1011.00,1,1,1,1,1,1,1
11-1021.91,4,4,4,4,4,4,4
11-1021.92,293,293,294,294,294,294,294
11-2021.91,1,1,1,1,1,1,1
11-2021.92,1,1,1,1,1,1,1


In [54]:
posting_df_MSA = posting_df.groupby(['MSA_ID']).count()
posting_df_MSA.head()

,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID,OCC_CODE
MSA_ID,,,,,,,
10420.0,15,15,15,15,15,15,15
10900.0,12,12,12,12,12,12,12
12060.0,126,126,126,126,126,126,126
12420.0,90,90,90,90,90,90,90
12580.0,20,20,20,20,20,20,20


In [55]:
#Perform Groubys on msa_df
msa_df_OCC = msa_df.groupby(['OCC_CODE']).mean()
msa_df_OCC.head()

,MSA_ID,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
OCC_CODE,,,,,,,,,,
11-1011,29914.000000,92.666667,1.654000,1.181333,105342.666667,53819.333333,68726.666667,93689.333333,123424.666667,173630.000000
11-1021,31450.996169,2082.260536,14.993027,0.917471,96114.444444,39498.160920,56049.501916,80143.524904,118010.574713,172455.478927
11-1031,31091.719745,143.184713,0.574662,1.614522,44833.375796,25077.388535,28214.203822,36569.681529,52524.203822,77397.898089
11-2011,36955.952381,153.095238,0.149881,0.876667,103666.428571,53591.428571,71100.000000,94747.619048,125297.619048,164601.190476
11-2021,33202.287582,339.803922,1.028876,0.572876,113340.457516,62261.830065,78638.888889,103665.294118,137923.333333,178686.470588


In [56]:
msa_df_MSA = msa_df.groupby(['MSA_ID']).mean()
msa_df_MSA.head()

,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
MSA_ID,,,,,,,,,
10180,256.517857,3.851402,1.314375,46104.776786,29557.455357,35431.428571,43662.142857,54407.678571,67049.017857
10380,257.872340,5.659014,1.862553,30245.390071,20116.170213,23464.468085,28356.808511,35213.404255,43742.269504
10420,746.925000,2.266470,1.190975,52309.375000,33143.050000,40158.475000,50064.025000,62337.450000,75381.650000
10500,255.611111,4.335828,1.305111,46614.722222,28690.277778,35762.333333,44932.277778,55331.833333,67193.888889
10540,177.570093,3.939005,2.234813,51859.719626,35119.158879,41025.654206,50036.775701,60816.962617,72574.532710


In [57]:
print(national_df_LONG['Non-Entry Level'].count())
print(national_df_OCC['Non-Entry Level'].count())
print(skill_df_LONG['Artistic_oi'].count())
print(skill_df_OCC['Artistic_oi'].count())
print(posting_df_OCC['MSA_ID'].count())
print(posting_df_MSA['YEAR'].count())
print(posting_df_LONG_OCC['YEAR'].count())
print(msa_df_OCC['Total Employment'].count())
print(msa_df_MSA['Total Employment'].count())

292
236
967
773
76
60
85
769
396


In [58]:
#######
#Merge#
#######

In [59]:
#Create DF to analyze postings per position
posting_position_freq = posting_df_LONG_OCC.merge(LONG_OCC_TITLE, on='LONG_OCC_CODE', how='inner', left_index=False, right_index=True)
posting_position_freq.head(30)

,MSA_ID,CITY,STATE,YEAR,BLS Job Title_x,JOB_ID,OCC_CODE,BLS Job Title_y
LONG_OCC_CODE,,,,,,,,
11-1011.00,1,1,1,1,1,1,1,Chief Executive Officer
11-1021.91,4,4,4,4,4,4,4,Director of Operations
11-1021.92,293,293,294,294,294,294,294,General Manager
11-2021.91,1,1,1,1,1,1,1,Vice President of Marketing
11-2021.92,1,1,1,1,1,1,1,Software Product Manager
11-2022.00,5,5,5,5,5,5,5,Sales Manager
11-3011.91,4,4,4,4,4,4,4,Facilities Manager
11-3031.00,1,1,1,1,1,1,1,Accounting Manager
11-3031.92,1,1,1,1,1,1,1,Financial Reporting Manager


In [60]:
posting_position_freq[['Job Title','Job Count']] = posting_position_freq[['BLS Job Title_y','JOB_ID']]
posting_position_freq.head()

,MSA_ID,CITY,STATE,YEAR,BLS Job Title_x,JOB_ID,OCC_CODE,BLS Job Title_y,Job Title,Job Count
LONG_OCC_CODE,,,,,,,,,,
11-1011.00,1,1,1,1,1,1,1,Chief Executive Officer,Chief Executive Officer,1
11-1021.91,4,4,4,4,4,4,4,Director of Operations,Director of Operations,4
11-1021.92,293,293,294,294,294,294,294,General Manager,General Manager,294
11-2021.91,1,1,1,1,1,1,1,Vice President of Marketing,Vice President of Marketing,1
11-2021.92,1,1,1,1,1,1,1,Software Product Manager,Software Product Manager,1


In [61]:
posting_position_freq = posting_position_freq.drop(columns = ['MSA_ID','STATE','CITY','BLS Job Title_x','OCC_CODE','BLS Job Title_y','JOB_ID'])
posting_position_freq.head()

,YEAR,Job Title,Job Count
LONG_OCC_CODE,,,
11-1011.00,1,Chief Executive Officer,1
11-1021.91,4,Director of Operations,4
11-1021.92,294,General Manager,294
11-2021.91,1,Vice President of Marketing,1
11-2021.92,1,Software Product Manager,1


In [62]:
posting_position_freq = posting_position_freq.reset_index()
posting_position_freq.head()

,LONG_OCC_CODE,YEAR,Job Title,Job Count
0,11-1011.00,1,Chief Executive Officer,1
1,11-1021.91,4,Director of Operations,4
2,11-1021.92,294,General Manager,294
3,11-2021.91,1,Vice President of Marketing,1
4,11-2021.92,1,Software Product Manager,1


In [63]:
#Create DF to analyze postings per area
posting_location_msa = posting_df_MSA.merge(msa_df_MSA, how='inner', left_index=True, right_index=True)
posting_location_msa.head()

,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID,OCC_CODE,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
MSA_ID,,,,,,,,,,,,,,,,
10420,15,15,15,15,15,15,15,746.925000,2.266470,1.190975,52309.375000,33143.050000,40158.475000,50064.025000,62337.450000,75381.650000
10900,12,12,12,12,12,12,12,790.160920,2.160506,1.034046,55231.195402,34564.735632,41953.379310,52518.597701,65800.022989,80668.275862
12060,126,126,126,126,126,126,126,4333.420139,1.579517,0.958003,53734.340278,32003.090278,40037.552083,50711.562500,64414.253472,80277.135417
12420,90,90,90,90,90,90,90,1990.000000,1.853884,1.133870,56116.109980,34547.393075,42184.704684,53026.517312,67040.957230,82413.136456
12580,20,20,20,20,20,20,20,2271.857143,1.651025,1.108429,59450.392857,36648.375000,44898.625000,56726.017857,71147.839286,86812.571429


In [64]:
#Drop unnecessary fields
posting_location_msa = posting_location_msa.drop(columns = ['CITY','STATE','YEAR','LONG_OCC_CODE','BLS Job Title','OCC_CODE'])
posting_location_msa.head()

,JOB_ID,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
MSA_ID,,,,,,,,,,
10420,15,746.925000,2.266470,1.190975,52309.375000,33143.050000,40158.475000,50064.025000,62337.450000,75381.650000
10900,12,790.160920,2.160506,1.034046,55231.195402,34564.735632,41953.379310,52518.597701,65800.022989,80668.275862
12060,126,4333.420139,1.579517,0.958003,53734.340278,32003.090278,40037.552083,50711.562500,64414.253472,80277.135417
12420,90,1990.000000,1.853884,1.133870,56116.109980,34547.393075,42184.704684,53026.517312,67040.957230,82413.136456
12580,20,2271.857143,1.651025,1.108429,59450.392857,36648.375000,44898.625000,56726.017857,71147.839286,86812.571429


In [65]:
posting_location_ID = msa_Only_DF.merge(posting_location_msa, on='MSA_ID', how='inner', left_index=False, right_index=True)
posting_location_ID.head()

,MSA_ID,Metro Area,JOB_ID,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
2,10420,"Akron, OH",15,746.925000,2.266470,1.190975,52309.375000,33143.050000,40158.475000,50064.025000,62337.450000,75381.650000
8,10900,"Allentown-Bethlehem-Easton, PA-NJ",12,790.160920,2.160506,1.034046,55231.195402,34564.735632,41953.379310,52518.597701,65800.022989,80668.275862
19,12060,"Atlanta-Sandy Springs-Roswell, GA",126,4333.420139,1.579517,0.958003,53734.340278,32003.090278,40037.552083,50711.562500,64414.253472,80277.135417
23,12420,"Austin-Round Rock, TX",90,1990.000000,1.853884,1.133870,56116.109980,34547.393075,42184.704684,53026.517312,67040.957230,82413.136456
25,12580,"Baltimore-Columbia-Towson, MD",20,2271.857143,1.651025,1.108429,59450.392857,36648.375000,44898.625000,56726.017857,71147.839286,86812.571429


In [66]:
print(posting_location_ID['MSA_ID'].count())
print(posting_location_msa['JOB_ID'].count())

56
56


In [67]:
#Rename JOB_ID to avoid confusion
posting_location_ID.rename(columns = {'JOB_ID':'Job Count'}, inplace = True)

In [68]:
#Verify Numbers after merge 8.735913143234185
check0 = posting_position_freq['Job Count'].sum()  
check1 = posting_location_ID['Job Count'].sum()  
print(f'Posting Frequency count: {check0}. Location Frequency count: {check1}')

Posting Frequency count: 5079. Location Frequency count: 5027


In [69]:
#Merge skill_df and postings_df_long_occ for kmeans
advance_opp = skill_df.merge(LONG_OCC_TITLE, on='LONG_OCC_CODE', how='inner', left_index=False, right_index=True)
advance_opp.head(30)

,LONG_OCC_CODE,BLS Job Title_x,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,...,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE,BLS Job Title_y
0,11-1011.00,Chief Executives,2.67,5.33,7.00,2.00,1.33,3.67,1.00,2.12,...,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011,Chief Executive Officer
7,11-2022.00,Sales Managers,2.00,4.67,7.00,2.00,3.00,3.67,1.38,2.00,...,3.83,4.52,5.33,5.67,4.67,4.00,5.33,5.50,11-2022,Sales Manager
28,11-9013.02,Farm and Ranch Managers,1.33,5.00,7.00,2.67,6.67,1.67,2.62,2.25,...,2.70,3.36,5.33,5.00,4.67,4.33,2.00,4.50,11-9013,Farm / Ranch Manager
30,11-9021.00,Construction Managers,1.33,4.67,7.00,2.00,5.67,2.00,1.38,2.75,...,3.48,4.16,5.33,5.67,4.67,5.00,5.33,5.83,11-9021,Construction Manager
42,11-9111.00,Medical and Health Services Managers,1.67,5.00,7.00,3.33,1.67,5.00,1.62,2.25,...,3.38,4.27,5.00,6.00,4.67,6.00,5.67,6.00,11-9111,Healthcare Administrator
60,13-1021.00,"Buyers and Purchasing Agents, Farm Products",1.67,6.00,6.33,1.33,4.00,2.67,1.88,2.00,...,3.59,4.69,4.00,5.33,3.67,4.00,3.67,4.17,13-1021,Procurement / Sourcing Specialist
65,13-1032.00,"Insurance Appraisers, Auto Damage",1.00,6.67,4.33,2.00,5.67,2.33,2.50,2.12,...,3.13,3.85,4.00,5.00,3.67,4.33,5.00,4.67,13-1032,Auto Damage Appraiser
72,13-1051.00,Cost Estimators,1.00,6.00,6.00,2.67,3.00,1.33,1.12,1.88,...,3.05,4.23,4.00,4.67,4.00,4.67,3.67,4.50,13-1051,Estimator
76,13-1081.00,Logisticians,1.33,5.67,7.00,2.33,2.33,3.33,1.00,2.00,...,3.46,4.21,5.33,5.67,5.00,5.00,4.33,4.83,13-1081,Logistician
79,13-1111.00,Management Analysts,1.67,4.33,6.00,6.33,1.33,2.33,1.00,1.88,...,3.73,4.46,5.33,5.33,5.00,6.00,3.67,5.17,13-1111,Program Analyst


In [70]:
#Clean advance_opp
advance_opp = advance_opp.drop(columns=['BLS Job Title_x'])
#advance_opp = advance_opp.reset_index(drop = True)
advance_opp.head()

,LONG_OCC_CODE,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,Category Flexibility_im,...,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE,BLS Job Title_y
0,11-1011.00,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,3.50,...,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011,Chief Executive Officer
7,11-2022.00,2.00,4.67,7.0,2.00,3.00,3.67,1.38,2.00,3.25,...,3.83,4.52,5.33,5.67,4.67,4.00,5.33,5.50,11-2022,Sales Manager
28,11-9013.02,1.33,5.00,7.0,2.67,6.67,1.67,2.62,2.25,3.12,...,2.70,3.36,5.33,5.00,4.67,4.33,2.00,4.50,11-9013,Farm / Ranch Manager
30,11-9021.00,1.33,4.67,7.0,2.00,5.67,2.00,1.38,2.75,3.25,...,3.48,4.16,5.33,5.67,4.67,5.00,5.33,5.83,11-9021,Construction Manager
42,11-9111.00,1.67,5.00,7.0,3.33,1.67,5.00,1.62,2.25,3.50,...,3.38,4.27,5.00,6.00,4.67,6.00,5.67,6.00,11-9111,Healthcare Administrator


In [71]:
#Rename BLS Job Title_y
advance_opp['Job Title'] = advance_opp['BLS Job Title_y']
advance_opp.head()

,LONG_OCC_CODE,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,Category Flexibility_im,...,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE,BLS Job Title_y,Job Title
0,11-1011.00,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,3.50,...,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011,Chief Executive Officer,Chief Executive Officer
7,11-2022.00,2.00,4.67,7.0,2.00,3.00,3.67,1.38,2.00,3.25,...,4.52,5.33,5.67,4.67,4.00,5.33,5.50,11-2022,Sales Manager,Sales Manager
28,11-9013.02,1.33,5.00,7.0,2.67,6.67,1.67,2.62,2.25,3.12,...,3.36,5.33,5.00,4.67,4.33,2.00,4.50,11-9013,Farm / Ranch Manager,Farm / Ranch Manager
30,11-9021.00,1.33,4.67,7.0,2.00,5.67,2.00,1.38,2.75,3.25,...,4.16,5.33,5.67,4.67,5.00,5.33,5.83,11-9021,Construction Manager,Construction Manager
42,11-9111.00,1.67,5.00,7.0,3.33,1.67,5.00,1.62,2.25,3.50,...,4.27,5.00,6.00,4.67,6.00,5.67,6.00,11-9111,Healthcare Administrator,Healthcare Administrator


In [72]:
#Drop BLS Job Title_y
advance_opp = advance_opp.drop(columns=['BLS Job Title_y'])
advance_opp.head()

,LONG_OCC_CODE,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,Category Flexibility_im,...,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE,Job Title
0,11-1011.00,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,3.50,...,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011,Chief Executive Officer
7,11-2022.00,2.00,4.67,7.0,2.00,3.00,3.67,1.38,2.00,3.25,...,3.83,4.52,5.33,5.67,4.67,4.00,5.33,5.50,11-2022,Sales Manager
28,11-9013.02,1.33,5.00,7.0,2.67,6.67,1.67,2.62,2.25,3.12,...,2.70,3.36,5.33,5.00,4.67,4.33,2.00,4.50,11-9013,Farm / Ranch Manager
30,11-9021.00,1.33,4.67,7.0,2.00,5.67,2.00,1.38,2.75,3.25,...,3.48,4.16,5.33,5.67,4.67,5.00,5.33,5.83,11-9021,Construction Manager
42,11-9111.00,1.67,5.00,7.0,3.33,1.67,5.00,1.62,2.25,3.50,...,3.38,4.27,5.00,6.00,4.67,6.00,5.67,6.00,11-9111,Healthcare Administrator


In [73]:
#Reset Index
advance_opp = advance_opp.reset_index(drop = True)
advance_opp.head()

,LONG_OCC_CODE,Artistic_oi,Conventional_oi,Enterprising_oi,Investigative_oi,Realistic_oi,Social_oi,Arm-Hand Steadiness_im,Auditory Attention_im,Category Flexibility_im,...,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE,Job Title
0,11-1011.00,2.67,5.33,7.0,2.00,1.33,3.67,1.00,2.12,3.50,...,3.67,4.75,6.33,7.00,7.00,5.00,5.33,6.33,11-1011,Chief Executive Officer
1,11-2022.00,2.00,4.67,7.0,2.00,3.00,3.67,1.38,2.00,3.25,...,3.83,4.52,5.33,5.67,4.67,4.00,5.33,5.50,11-2022,Sales Manager
2,11-9013.02,1.33,5.00,7.0,2.67,6.67,1.67,2.62,2.25,3.12,...,2.70,3.36,5.33,5.00,4.67,4.33,2.00,4.50,11-9013,Farm / Ranch Manager
3,11-9021.00,1.33,4.67,7.0,2.00,5.67,2.00,1.38,2.75,3.25,...,3.48,4.16,5.33,5.67,4.67,5.00,5.33,5.83,11-9021,Construction Manager
4,11-9111.00,1.67,5.00,7.0,3.33,1.67,5.00,1.62,2.25,3.50,...,3.38,4.27,5.00,6.00,4.67,6.00,5.67,6.00,11-9111,Healthcare Administrator


In [74]:
#Check DF
msa_df_MSA.head()

,Total Employment,Jobs per 1000,Location Quotient,Mean Annual Wage,Annual Wage 10th Pct,Annual Wage 25th Pct,Median Annual Wage,Annual Wage 75th Pct,Annual Wage 90th Pct
MSA_ID,,,,,,,,,
10180,256.517857,3.851402,1.314375,46104.776786,29557.455357,35431.428571,43662.142857,54407.678571,67049.017857
10380,257.872340,5.659014,1.862553,30245.390071,20116.170213,23464.468085,28356.808511,35213.404255,43742.269504
10420,746.925000,2.266470,1.190975,52309.375000,33143.050000,40158.475000,50064.025000,62337.450000,75381.650000
10500,255.611111,4.335828,1.305111,46614.722222,28690.277778,35762.333333,44932.277778,55331.833333,67193.888889
10540,177.570093,3.939005,2.234813,51859.719626,35119.158879,41025.654206,50036.775701,60816.962617,72574.532710


In [75]:
#Check DF
posting_df.head()

,MSA_ID,CITY,STATE,YEAR,LONG_OCC_CODE,BLS Job Title,JOB_ID,OCC_CODE
0,42660.0,Seattle,WA,2019,13-1071.91,Recruiter (General),38472841335,13-1071
1,38060.0,Chandler,AZ,2019,13-1051.00,Estimator,38474073426,13-1051
2,38060.0,Chandler,AZ,2019,49-3021.00,Auto Body Technician,38474083740,49-3021
3,45300.0,Tampa,FL,2020,11-1021.92,General Manager,38709992297,11-1021
4,41700.0,San Antonio,TX,2020,49-9098.00,Maintenance Helper / Assistant,38709918452,49-9098


In [76]:
#Verify lengths
advance_opp_len = advance_opp["LONG_OCC_CODE"].count()
posting_location_msa_len = posting_location_msa["JOB_ID"].count()
print(f'advance_opp has {advance_opp_len}, posting_location_msa has {posting_location_msa_len}')

advance_opp has 58, posting_location_msa has 56


In [119]:
#Merge national_df and advance_opp for kmeans
advance_opp_ML = national_df.merge(advance_opp, on='LONG_OCC_CODE', how='inner', left_index=False, right_index=True)
advance_opp_ML.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage,...,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE_y,Job Title
8,Software Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
9,Java Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
11,.NET Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
14,Application Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
16,Solutions Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer


In [120]:
#Reset Index
advance_opp_ML = advance_opp_ML.reset_index(drop=True)
advance_opp_ML.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage,...,Social Orientation_im,Stress Tolerance_im,Achievement_ex,Independence_ex,Recognition_ex,Relationships_ex,Support_ex,Working Conditions_ex,OCC_CODE_y,Job Title
0,Software Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
1,Java Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
2,.NET Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
3,Application Developer / Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer
4,Solutions Engineer,15-1131.00,25.7,74.3,Bachelor's degree,None,False,True,False,False,...,3.21,3.9,5.67,5.0,4.33,2.33,5.0,5.5,15-1131,Senior Software Developer / Engineer


In [109]:
national_df.head()

,BLS Job Title,LONG_OCC_CODE,Percentage with AA Degree or Less,Percentage with BA Degree or More,Typical education needed for entry,Work experience in a related occupation,Non-Entry Level,BA Entry Level,Middle Skilled,Low Wage,OCC_CODE
0,Computer Operator,43-9011.00,64.9,35.1,High school diploma or equivalent,None,False,False,True,False,43-9011
1,Business Intelligence Analyst,15-1199.93,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
2,IT Project Manager,15-1199.95,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
3,Network Engineer / Architect,15-1199.02,43.8,56.4,Bachelor's degree,None,False,True,False,False,15-1199
4,Network Engineer / Architect,15-1133.00,14.7,85.2,Bachelor's degree,None,False,True,False,False,15-1133


In [77]:
#########
#Analyze#
#########

In [78]:
diff=check0-check1
per_change=diff/check0*100
print(per_change)

1.0238235873203387


In [79]:
#Sort DF to show titles with most postings
posting_position_freq = posting_position_freq.sort_values(by='Job Count', ascending=False)
posting_position_freq.head(20)

,LONG_OCC_CODE,YEAR,Job Title,Job Count
72,49-3021.00,2068,Auto Body Technician,2068
84,53-7061.00,549,Detailer,549
15,13-1032.00,539,Auto Damage Appraiser,539
2,11-1021.92,294,General Manager,294
68,43-6014.00,255,Administrative Clerk / Coordinator,255
52,37-2011.00,203,Janitor / Cleaner,203
16,13-1051.00,175,Estimator,175
62,43-4051.00,171,Customer Service Representative (General),171
56,41-2031.00,162,Retail Sales Associate (General),162
77,49-9098.00,110,Maintenance Helper / Assistant,110


In [80]:
#Print the total amount of positions posted
print(f'Total positions posted: {posting_position_freq["Job Title"].count()}')

Total positions posted: 85


In [81]:
#Re-Sort DF to chart titles with most postings
posting_position_freq = posting_position_freq.sort_values(by='Job Count', ascending=True)
posting_position_freq.head()

,LONG_OCC_CODE,YEAR,Job Title,Job Count
0,11-1011.00,1,Chief Executive Officer,1
10,11-9021.00,1,Construction Manager,1
14,13-1021.00,1,Procurement / Sourcing Specialist,1
12,11-9111.94,1,Medical Director (General),1
31,13-2053.00,1,Senior Insurance Underwriter,1


In [102]:
#Plot bar chart to show distrobution of job postings
data = px.data.gapminder()

fig = px.bar(posting_position_freq, x='Job Title', y='Job Count',
             hover_data=['LONG_OCC_CODE','Job Title', 'Job Count'],
             title="Postings per position",
             labels={'LONG_OCC_CODE':'Long OCC Code'},
             color='Job Count',
             height=700)
fig.show()
print(f'Total positions posted: {posting_position_freq["Job Title"].count()}')

Total positions posted: 85


In [83]:
#Show positions with 100 or more postings
posting_position_100 = posting_position_freq.loc[posting_position_freq['Job Count'] >= 100]
posting_position_100 = posting_position_100.sort_values(by='Job Count', ascending=False)
posting_position_100.head(20)

,LONG_OCC_CODE,YEAR,Job Title,Job Count
72,49-3021.00,2068,Auto Body Technician,2068
84,53-7061.00,549,Detailer,549
15,13-1032.00,539,Auto Damage Appraiser,539
2,11-1021.92,294,General Manager,294
68,43-6014.00,255,Administrative Clerk / Coordinator,255
52,37-2011.00,203,Janitor / Cleaner,203
16,13-1051.00,175,Estimator,175
62,43-4051.00,171,Customer Service Representative (General),171
56,41-2031.00,162,Retail Sales Associate (General),162
77,49-9098.00,110,Maintenance Helper / Assistant,110


In [101]:
#Plot positions with over 100 postings
posting_position_100 = posting_position_100.sort_values(by='Job Count', ascending=True)
data = px.data.gapminder()

fig = px.bar(posting_position_100, x='Job Title', y='Job Count',
             hover_data=['LONG_OCC_CODE','Job Title', 'Job Count'],
             title="Positions with Over 100 Postings",
             labels={'LONG_OCC_CODE':'Long OCC Code'},
             color='Job Count', height=700)
fig.show()

In [85]:
#Sort DF to show metropolitan areas with most postings
posting_location_ID = posting_location_ID.sort_values(by='Job Count', ascending=False)
posting_location_ID = posting_location_ID[['MSA_ID','Metro Area', 'Job Count']]
posting_location_ID = posting_location_ID.reset_index(drop=True)
posting_location_ID.head(10)

,MSA_ID,Metro Area,Job Count
0,19100,"Dallas-Fort Worth-Arlington, TX",859
1,16980,"Chicago-Naperville-Elgin, IL-IN-WI",352
2,38060,"Phoenix-Mesa-Scottsdale, AZ",288
3,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",264
4,41700,"San Antonio-New Braunfels, TX",238
5,26420,"Houston-The Woodlands-Sugar Land, TX",216
6,31080,"Los Angeles-Long Beach-Anaheim, CA",192
7,42660,"Seattle-Tacoma-Bellevue, WA",192
8,19820,"Detroit-Warren-Dearborn, MI",168
9,19740,"Denver-Aurora-Lakewood, CO",143


In [86]:
print(f'Total locations with postings: {posting_location_ID["Metro Area"].count()}')

Total locations with postings: 56


In [99]:
#Plot bar chart to show distrobution of job postings per metro area
posting_location_ID = posting_location_ID.sort_values(by='Job Count', ascending=True)
data = px.data.gapminder()

fig = px.bar(posting_location_ID, x='Metro Area', y='Job Count',
             hover_data=['MSA_ID','Metro Area', 'Job Count'],
             title="Postings per Location",
             labels={'MSA_ID':'MSA ID'},
             color='Job Count', height=700)
fig.show()
print(f'Total locations with postings: {posting_location_ID["Metro Area"].count()}')

Total locations with postings: 56


In [88]:
posting_location_100 = posting_location_ID.loc[posting_location_ID['Job Count'] >= 100]
posting_location_100 = posting_location_100.sort_values(by='Job Count', ascending=False)
posting_location_100.head(100)

,MSA_ID,Metro Area,Job Count
0,19100,"Dallas-Fort Worth-Arlington, TX",859
1,16980,"Chicago-Naperville-Elgin, IL-IN-WI",352
2,38060,"Phoenix-Mesa-Scottsdale, AZ",288
3,34980,"Nashville-Davidson--Murfreesboro--Franklin, TN",264
4,41700,"San Antonio-New Braunfels, TX",238
5,26420,"Houston-The Woodlands-Sugar Land, TX",216
7,42660,"Seattle-Tacoma-Bellevue, WA",192
6,31080,"Los Angeles-Long Beach-Anaheim, CA",192
8,19820,"Detroit-Warren-Dearborn, MI",168
9,19740,"Denver-Aurora-Lakewood, CO",143


In [98]:
#Plot bar chart to show distrobution of job postings of atleast 100 per metro area
posting_location_100 = posting_location_100.sort_values(by='Job Count', ascending=True)
data = px.data.gapminder()

fig = px.bar(posting_location_100, x='Metro Area', y='Job Count',
             hover_data=['MSA_ID','Metro Area', 'Job Count'],
             title="Locations with Over 100 Postings",
             labels={'MSA_ID':'MSA ID'},
             color='Job Count', height=700)
fig.show()